In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from datetime import datetime
from bs4 import BeautifulSoup

keyword = ''
url = 'https://www.youtube.com/results?search_query={}'.format(keyword)

driver = webdriver.Chrome()
driver.get(url)

# 스크롤 n번 반복
for _ in range(10):
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(2)  # 스크롤 후 잠시 대기

soup = bs(driver.page_source, 'html.parser')

video_url = soup.select('a#video-title')

url_list = []


for i in range(len(video_url)):
    video_url_str = 'https://www.youtube.com' + video_url[i].get('href')
    if 'shorts' in video_url_str.lower():
        continue  # "shorts"가 포함된 동영상 주소는 건너뜁니다.
    url_list.append(video_url_str)
    
youtubeDic = {
    '주소': url_list,
}


name_list = []
view_list = []
channel_list = []
subscriber_list = []
hashtag_list = []
comment_list = []
video_length_list = []

for i, url in enumerate(url_list):
    driver.get(url)
    time.sleep(3)

    # 페이지 스크롤 다운
    body = driver.find_element(By.TAG_NAME, 'body')
    num_of_pagedowns = 3
    while num_of_pagedowns:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
        num_of_pagedowns -= 1

    soup = bs(driver.page_source, 'html.parser')
    
    # 영상제목 크롤링
    try:
        name = soup.select('yt-formatted-string.style-scope.ytd-watch-metadata')[0].text.strip()
        name_list.append(name)
    except:
        name_list.append('')

    # 채널명 크롤링
    try:
        channel= soup.select('a.yt-simple-endpoint.style-scope.yt-formatted-string')[0].text.strip()
        channel_list.append(channel)
    except:
        channel_list.append('')

    # 구독자수 크롤링
    try:
        subscriber = soup.select('yt-formatted-string#owner-sub-count.style-scope.ytd-video-owner-renderer')[0].text.strip()
        subscriber_list.append(subscriber)
    except:
        subscriber_list.append('')


    # 조회수 크롤링
    try:
        view = soup.select('span.bold.style-scope.yt-formatted-string')[0]
        view_list.append(view.text.strip().replace('조회수 ', '').replace('회', ''))
    except:
        view_list.append('')

    # 해시태그 크롤링
    try:
        hashtags = soup.select('a.yt-simple-endpoint.style-scope.yt-formatted-string.bold')
        hashtag = [hashtag.text.strip() for hashtag in hashtags]
    except:
        hashtag = []
    hashtag_list.append(hashtag)

    # 댓글 크롤링
    try:
        comments = soup.select('yt-formatted-string#content-text')
        comment = [comment.text.strip() for comment in comments]
    except:
        comment = []
    comment_list.append(comment)
    
       
    # 영상길이 크롤링   # <span class="ytp-time-duration">22:11</span> #예외처리
    try:
        video_length = soup.select('span.ytp-time-duration')[0].text.strip()
        video_length_list.append(video_length)
    except:
        video_length_list.append('')



    
driver.close()

youtubeDic2 = {
    '제목': name_list, 
    '주소': url_list, 
    '조회수': view_list, 
    '채널명': channel_list,
    '구독자수': subscriber_list,
    '해시태그': hashtag_list,
    '댓글': comment_list,
    '영상길이': video_length_list, 
}

print('name_list:', len(name_list))
print('url_list:', len(url_list))
print('view_list:', len(view_list))
print('channel_list:', len(channel_list))
print('subscriber_list:', len(subscriber_list))
print('hashtag_list:', len(hashtag_list))
print('comment_list:', len(comment_list))
print('video_length_list:', len(video_length_list))

youtubeDf = pd.DataFrame(youtubeDic2)
print(youtubeDf)
youtubeDf.to_excel(r'C:\Users\82106\Desktop\BOAZ\유튜브크롤링요리시도.xlsx', index=False)